In [1]:
import pandas as pd

In [3]:
# load data
DATA_FILE_PATH = 'insert path here'
DATA_FILE_NAME_GLOBAL = 'GlobalTemperatures.csv'
DATA_FILE_NAME_CITY = 'GlobalTemperaturesByCity.csv'

df_global = pd.read_csv(DATA_FILE_PATH + DATA_FILE_NAME_GLOBAL)
df_global['dt'] = pd.to_datetime(df_global['dt'], format='%Y-%m-%d')

df_per_city = pd.read_csv(DATA_FILE_PATH + DATA_FILE_NAME_CITY)
df_per_city['dt'] = pd.to_datetime(df_per_city['dt'], format='%Y-%m-%d')

Can global warming be observed on earth's temperature evolution?

In [26]:
# average for each year
average_temp_per_year_df = pd.DataFrame({ "year": [], "land_temp_per_year": [], "land_and_ocean_temp_per_year": []})

years = list(range(1750, 2015+1))

for year in years:
    filtered_df = df_global.loc[(df['dt'] >= str(year) + '-01-01')
                     & (df_global['dt'] < str(year) +'-12-02')]
    mean_land = filtered_df.LandAverageTemperature.mean()
    mean_land_and_ocean = filtered_df.LandAndOceanAverageTemperature.mean(skipna=True)
    average_temp_per_year_df.loc[len(average_temp_per_year_df)] = [year, mean_land, mean_land_and_ocean]

display(average_temp_per_year_df.sort_values(by=['year']))


,year,land_temp_per_year,land_and_ocean_temp_per_year
0,1750.0,8.719364,NaN
1,1751.0,7.976143,NaN
2,1752.0,5.779833,NaN
3,1753.0,8.388083,NaN
4,1754.0,8.469333,NaN
...,...,...,...
261,2011.0,9.516000,15.769500
262,2012.0,9.507333,15.802333
263,2013.0,9.606500,15.854417
264,2014.0,9.570667,15.913000


Can the average country temperature be plotted in a compact way? you
may take a (logical) sample of countries. You may also get the year’s
average temperature for each.

In [ ]:
# get average temperature per country
df_per_city